In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd
import statsmodels.api as sm

import sys
sys.path.append("../")
from bargains_logit import * 

In [2]:
COST =5
WTP =  np.array([25,25])
MC = np.array([0,0])

print(simult_bargain(15,15,COST,WTP,MC,active=True))
print(seq_bargain(15, COST, WTP, MC))

#(array([15.86215466]), array([15.6463623]))
#(array([16.12045556]), array([17.17686621]))

(array([15.86215466]), array([15.6463623]))
(array([16.12045556]), array([17.17686621]))


In [32]:
def run_test0(df,print_v=False):
    df = df[ (df['s1']!=0) & (df['s1']!=1) &  (df['phi1']>=0) & (df['phi2']>=0) ] #(np.abs(df['phi1'])<=100) & (np.abs(df['phi2'])<=100) ]
    df = df.reindex()
    df['s0'] = 1 - df['s1'] - df['s2']
    df['log(s0)'] = np.log(df['s0'])
    df['s0^2'] = df['s0']**2
    df['1/s0'] = 1/df['s0']
    df['s1*phi1'] = df['s1']*df['phi1']
    df['pi_h'] = df['s2']*df['phi2'] + df['s1']*df['phi1']
    #clean df of weird obs
    res0= sm.OLS(df['phi2'],sm.add_constant(df[['pi_h','s1*phi1','v']])).fit()
    
    print(res0.summary())
    
    #clean_resid = np.abs(res0.resid) < 3.5*res0.resid.std()
    #df_clean = df[clean_resid]
    
    #res2 = sm.OLS(df_clean['phi2'],sm.add_constant(df_clean[['v','s0']])).fit()

    return (res0.params[1], res0.bse[1], res0.pvalues[1])

passive_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/passive_data_0.csv',index_col=0)
active_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/active_data_0.csv',index_col=0)
seq_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/seq_data_0.csv',index_col=0)

print(seq_result1)

dfs = [passive_result1,active_result1,seq_result1]

for i in range(3): 
    df = dfs[i]
    df_sample = df
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print(run_test0(df_sample))

         phi1          phi2         p1          p2            s1        s2  \
0   16.620496 -5.545971e+28  17.328442 -200.537958  1.192244e-19  1.000000   
0   16.049381  1.707679e+01  22.583934   22.741258  3.676505e-01  0.356263   
0   15.875391  1.752073e+01  22.564684   23.944019  4.061480e-01  0.308231   
0   16.384125  1.823080e+01  22.005985   23.329097  4.172944e-01  0.320272   
0   15.133662 -5.166952e+28  19.488032 -200.537958  7.740792e-20  1.000000   
..        ...           ...        ...         ...           ...       ...   
0   16.870626  1.879042e+01  23.385786   24.794242  4.359024e-01  0.328891   
0   15.815404  1.781559e+01  25.286438   23.125665  2.837559e-01  0.437147   
0   16.127544  1.774665e+01  23.320714   24.051665  3.955940e-01  0.341791   
0   16.963543  1.863731e+01  23.150137   24.869950  4.468862e-01  0.316822   
0   16.217303 -5.303623e+28  20.625649 -200.537958  6.165569e-20  1.000000   

          x1        x2          v  
0  -4.297843 -1.098649  24.

In [ ]:
def gen_data(params,num_mkts=20,nobs=100,var=5):
    wtp,cost,beta1,beta2,emc1,emc2=params
    emc = emc1,emc2
    passive_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','x1','x2','v']) 
    active_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','x1','x2','v'])
    seq_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','x1','x2','v'])
    for j in range(0,num_mkts):
        mkt_wtp = np.random.normal(loc=0, scale=1)
        wtp_j = np.array([wtp+mkt_wtp,wtp+mkt_wtp])
        for i in range(0,nobs):
            #sequential
            mc1 = np.random.normal(loc=0, scale=var, size=(2,)) + np.array([emc1,emc2])
            mc2 = np.random.normal(loc=0, scale=1, size=(2,)) 
            mc = mc1+mc2

            #active results
            phi1,phi2 =simult_bargain(15,15, cost,wtp_j,mc, active=True)
            phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp_j,mc)

            row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','x1','x2','v'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],
                                                                                                mc1[1],wtp_j[0]]])
            active_result = pd.concat( [active_result, row] )
            #print('active_result',[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1],wtp_j[0]]])

            #passive results
            phi1,phi2 =simult_bargain(8,8, cost,wtp_j,mc, active=False)
            phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp_j,mc)
            row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','x1','x2','v'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],
                                                                                                mc1[1],wtp_j[0]]])
            passive_result = pd.concat( [passive_result, row] )
            #print('passive_result',[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1],wtp_j[0]]])

            
            #seq results
            phi1,phi2 = seq_bargain(15,cost,wtp_j,mc)
            phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp_j,mc)
            row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','x1','x2','v'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],
                                                                                                mc1[1],wtp_j[0]]])
            seq_result = pd.concat( [seq_result, row] )
            #print('seq_result',[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1],wtp_j[0]]])
            
    return seq_result, active_result,passive_result


#param_list = [[25,5,.5,.5,0,0],[23,4,.5,.5,0,0],[27,7,.5,.5,0,0],[25,5,.5,.1,0,0],[25,5,.1,.5,0,0]] old list 1/25
param_list = [[25,5,.5,.5,0,0]]#,[20,5,.5,.5,0,0],[30,7,.5,.5,0,0],[25,5,.5,.1,0,0],[25,5,.1,.5,0,0]]

for i in range(len(param_list)):
    seq_result,active_result,passive_result = gen_data(param_list[i],num_mkts=100,nobs=2,var=1)
    seq_result.to_csv('fake_data_wtp_logit/seq_data_%s.csv'%i)
    active_result.to_csv('fake_data_wtp_logit/active_data_%s.csv'%i)
    passive_result.to_csv('fake_data_wtp_logit/passive_data_%s.csv'%i)

In [33]:
def run_test(df,print_v=False):
    df = df.copy()
    df['phi1^2'] = df['phi1']**2
    df['1/phi1'] = 1/df['phi1']
    df['1/phi1^2'] = 1/(df['phi1']**2)
    df['log(phi1)'] = np.log(df['phi1'])
    
    #clean df of weird obs
    res0= sm.OLS(df['phi2'],sm.add_constant(df[['v','phi1','phi1^2','log(phi1)','1/phi1']])).fit()
    clean_resid = np.abs(res0.resid) < 3.5*res0.resid.std()
    df_clean = df[clean_resid]
    
    #regress
    res1= sm.OLS(df['phi2'],sm.add_constant(df[['v','phi1','phi1^2','log(phi1)','1/phi1']])).fit()
    res2 = sm.OLS(df_clean['phi2'],sm.add_constant(df_clean[['v']])).fit()
    if print_v :
        print(res2.summary())
        print(res1.summary())
    return np.array([res1.pvalues[1]<=.05,res2.pvalues[1]<=.05])


passive_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/passive_data_0.csv',index_col=0)
active_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/active_data_0.csv',index_col=0)
seq_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/seq_data_0.csv',index_col=0)

print(seq_result1)

dfs = [passive_result1,active_result1,seq_result1]
for i in range(3): 
    df = dfs[i]
    df_sample = df.sample(frac=0.5)
    if i==2:
        run_test(df_sample,print_v=True)
    print(run_test(df_sample))

         phi1          phi2         p1          p2            s1        s2  \
0   16.620496 -5.545971e+28  17.328442 -200.537958  1.192244e-19  1.000000   
0   16.049381  1.707679e+01  22.583934   22.741258  3.676505e-01  0.356263   
0   15.875391  1.752073e+01  22.564684   23.944019  4.061480e-01  0.308231   
0   16.384125  1.823080e+01  22.005985   23.329097  4.172944e-01  0.320272   
0   15.133662 -5.166952e+28  19.488032 -200.537958  7.740792e-20  1.000000   
..        ...           ...        ...         ...           ...       ...   
0   16.870626  1.879042e+01  23.385786   24.794242  4.359024e-01  0.328891   
0   15.815404  1.781559e+01  25.286438   23.125665  2.837559e-01  0.437147   
0   16.127544  1.774665e+01  23.320714   24.051665  3.955940e-01  0.341791   
0   16.963543  1.863731e+01  23.150137   24.869950  4.468862e-01  0.316822   
0   16.217303 -5.303623e+28  20.625649 -200.537958  6.165569e-20  1.000000   

          x1        x2          v  
0  -4.297843 -1.098649  24.

C:\Users\ericsc\AppData\Local\anaconda3\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


MissingDataError: exog contains inf or nans

In [ ]:
# Create an empty DataFrame to store the results
results_list = []

for j in range(5):
    # Read the dataframes
    seq_result = pd.read_csv('../GMM_est/fake_data_wtp/seq_data_%s.csv' % j, index_col=0)
    active_result = pd.read_csv('../GMM_est/fake_data_wtp/active_data_%s.csv' % j, index_col=0)
    passive_result = pd.read_csv('../GMM_est/fake_data_wtp/passive_data_%s.csv' % j, index_col=0)

    # Initialize rejection count variables
    rejections_seq = np.array([0,0])
    rejections_active = np.array([0,0])
    rejections_passive = np.array([0,0])

    nsims = 500
    
    for i in range(nsims):
        # Resample from seq dataframe
        subsample_seq = seq_result.sample(frac=0.5)
        result_seq = run_test(subsample_seq)
        rejections_seq = rejections_seq+ result_seq
        
        
        # Resample from active dataframe
        subsample_active = active_result.sample(frac=0.5)
        result_active = run_test(subsample_active)
        rejections_active = rejections_active+ result_active

        # Resample from passive dataframe
        subsample_passive = passive_result.sample(frac=0.5)
        result_passive = run_test(subsample_passive)
        rejections_passive = rejections_passive+ result_passive

    
    rejections_seq = rejections_seq/nsims
    rejections_active = rejections_active/nsims
    rejections_passive = rejections_passive/nsims
    

    
    # Add the results to the DataFrame
    results_list.append({'Round':j,'DataFrame': 'passive', 'Rejections 1': rejections_passive[0],
                         'Rejections 2': rejections_passive[1]})
    results_list.append({'Round':j,'DataFrame': 'active', 'Rejections 1': rejections_active[0],
                        'Rejections 2': rejections_active[1]})
    results_list.append({'Round':j,'DataFrame': 'seq', 'Rejections 1': rejections_seq[0],
                        'Rejections 2': rejections_seq[1]})

 
results_df = pd.DataFrame(data=results_list,columns=['Round','DataFrame','Rejections 1','Rejections 2'])

In [ ]:
print(results_df[['Round','DataFrame','Rejections 2','Rejections 1']])  

In [ ]:
#quickly adjust for the fact that the test is a 2 stage test... only run stage 2 if i reject passive...
results_df['Round 2'] = results_df['Rejections 1'].copy()
results_df.loc[results_df['DataFrame']=='passive','Round 2'] = (1  - results_df['Rejections 2'] 
                                                             + results_df['Rejections 1'] )[results_df['DataFrame']=='passive']

print(results_df[['Round','DataFrame','Rejections 2','Round 2']])  